In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 15
num_filter = 18
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 9s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 18, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 18, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 18
dropout_rate = 0.2
l = 15
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 18)   486         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 18)   72          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 18)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 4, 4, 72)     288         concatenate_52[0][0]             
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 4, 4, 72)     0           batch_normalization_56[0][0]     
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 4, 4, 9)      5832        activation_56[0][0]              
__________________________________________________________________________________________________
dropout_56 (Dropout)            (None, 4, 4, 9)      0           conv2d_57[0][0]                  
__________________________________________________________________________________________________
concatenate_53 (Concatenate)    (None, 4, 4, 81)     0           concatenate_52[0][0]             
          

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 301s 6ms/step - loss: 1.6393 - acc: 0.3920 - val_loss: 1.5626 - val_acc: 0.4544
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:44 - loss: 1.3242 - acc: 0.515750000/50000 [==============================] - 282s 6ms/step - loss: 1.2638 - acc: 0.5387 - val_loss: 1.5658 - val_acc: 0.5060
Epoch 3/50
45568/50000 [==========================>...] - ETA: 23s - loss: 1.0838 - acc: 0.607650000/50000 [==============================] - 285s 6ms/step - loss: 1.0771 - acc: 0.6103 - val_loss: 1.8674 - val_acc: 0.5000
Epoch 4/50
50000/50000 [==============================] - 283s 6ms/step - loss: 0.9753 - acc: 0.6496 - val_loss: 1.4441 - val_acc: 0.5719
Epoch 5/50
 3328/50000 [>.............................] - ETA: 4:08 - loss: 0.9304 - acc: 0.665950000/50000 [==============================] - 283s 6ms/step - loss: 0.9104 - acc: 0.6747 - val_loss: 1.3663 - val_acc: 0.5628
Epoch 

 1536/50000 [..............................] - ETA: 4:18 - loss: 0.3004 - acc: 0.897150000/50000 [==============================] - 283s 6ms/step - loss: 0.3040 - acc: 0.8921 - val_loss: 0.7198 - val_acc: 0.8139
Epoch 45/50
38528/50000 [======================>.......] - ETA: 1:01 - loss: 0.2993 - acc: 0.893750000/50000 [==============================] - 283s 6ms/step - loss: 0.3027 - acc: 0.8929 - val_loss: 0.7257 - val_acc: 0.8043
Epoch 46/50
50000/50000 [==============================] - 283s 6ms/step - loss: 0.2963 - acc: 0.8944 - val_loss: 0.6472 - val_acc: 0.8181
Epoch 47/50
 1536/50000 [..............................] - ETA: 4:17 - loss: 0.2695 - acc: 0.901050000/50000 [==============================] - 283s 6ms/step - loss: 0.2915 - acc: 0.8967 - val_loss: 0.8416 - val_acc: 0.7857
Epoch 48/50
38528/50000 [======================>.......] - ETA: 1:00 - loss: 0.2845 - acc: 0.897550000/50000 [==============================] - 283s 6ms/step - loss: 0.2885 - acc: 0.8966 - val_loss: 0.

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 24s 2ms/step
Test loss: 0.5563424746513367
Test accuracy: 0.8446


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')